# Overview

## How to Use this Notebook

This notebook shows you how to easily explore Terraform features and functions by directly using the OCI Comand Line Interface (CLI).  First you will need to have the CLI tool installed on your machine and working.  Then you add a few elements to your config file as noted below, then Authenticate to your tenancy using the notebook cell provided.


<div class="alert alert-block alert-warning">
<b>Example config file:</b> Use yellow boxes for examples that are not inside code cells, or use for mathematical formulas if needed. Typically also used to display warning messages.
</div>



This is <a href="https://en.wikipedia.org" title="Wikipedia">a reference</a>

this is a [click me](https://en.wikipedia.org "Testme")

Terraform [click me](https://docs.oracle.com/en-us/iaas/Content/ResourceManager/Tasks/managingstacksandjobs.htm#stackstate  "xme")



## References



this is a [click me](https://en.wikipedia.org "Testme")

Resource Manager Stacks and Jobs [link](https://docs.oracle.com/en-us/iaas/Content/ResourceManager/Tasks/managingstacksandjobs.htm#stackstate  "Stacks and Jobs")




# Initialize Connection to Tenancy
```python
x=1
print(x)
```


In [ ]:
# List Bastions
import oci
import pprint
import subprocess
import json ,  pprint
from IPython.display import display, Math, HTML, Markdown

tenancyProfile = 'tr-ash'
tenancyConfig = oci.config.from_file(profile_name=tenancyProfile)
securityProfile=  tenancyConfig['auth_profile']  
securityConfig = oci.config.from_file(profile_name=securityProfile)
region =  tenancyConfig['region']
token_file = securityConfig['security_token_file']
tenancyId = tenancyConfig['tenancy']
targetComptId = tenancyConfig['target_compartment']
# Config parameters
config = ' --profile ' + securityProfile +  ' --auth security_token'

display(HTML("<style>.container { width:75% !important; }</style>"))
# display(HTML("<style>div.output_scroll { height: 70em; }</style>"))


## General Use Methods

In [ ]:
# General Use Methods
def runCommand(pCommand):
    command=pCommand
    process = subprocess.Popen(command, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    out, err = process.communicate()
    return out.decode('utf-8'),  err.decode('utf-8')


## Authenticate Session

In [ ]:
command = 'oci session authenticate --region ' + tenancyConfig['region'] + \
        ' --profile ' + tenancyProfile + ' --profile-name ' + securityProfile
print('Creating Session for: ',securityProfile )
print('Running command: ',command )
stdout, stderr = runCommand(command)
print(stderr)

## Refresh Session

In [ ]:

command = 'oci session refresh --profile '  + securityProfile
print('Refreshing Session for: ',securityProfile )
print('Running command: ',command )
stdout, stderr = runCommand(command)
print(stderr)
command = 'oci session validate --profile ' + securityProfile
print('Validating Session for: ',securityProfile )
print('Running command: ',command )
stdout, stderr = runCommand(command)
print(stderr)

# Terraform CLI


## Stacks

### Stack List 

<div class="alert alert-block alert-success">
<b>Sample Output:</b> <p>
  There were 9  Stacks Found  <br>
CostApp-Ionut3  
   
    Description: Stack with var for API Gateway and monitor_trigger
    Stack ID: ocid1.ormstack.oc1.iad.aaa...
    Time Created:  2021-12-17T19:07:47.834000+00:00
    Status:  ACTIVE
    Terraform Version:  1.0.x
--  
    </p>
</div>

In [ ]:

command = 'oci resource-manager stack list  --compartment-id ' + targetComptId  + config
print('Command: \n', command)
stdout, stderr = runCommand(command)
print(stderr)
try:
    stacks = json.loads(stdout)
except:
    print( 'Json convert error' ,stdout)

print('There were ' + str(len(stacks['data']) )+ '  Stacks Found')
for stack in stacks['data']:
    print(stack['display-name'])
    print('    Description: '  ,stack['description'])
    print('    Stack ID: ',stack['id'])
    print('    Time Created: ', stack['time-created'])
    print('    Status: ', stack['lifecycle-state'])
    print('    Terraform Version: ', stack['terraform-version'])
    print('--')
# pprint.pprint(stack)



### Get Stack

In [ ]:
stackId = 'ocid1.ormstack.oc1.iad.aaaaaaaaddlrp4juzcgetgrqe2em2zryvnvasfe4gtwgjgv4n5hhdcjctova'
command = ' oci resource-manager stack get --stack-id  ' + stackId +  config
print('Command: \n', command)
stdout, stderr = runCommand(command)
print(stderr)
stack = ''
try:
    stack = json.loads(stdout)
    print(stack['data']['display-name'])
    print('    Git Branch: ',stack['data']['config-source']['branch-name'])
    print('    Git Directory: ',stack['data']['config-source']['working-directory'])
    print('    Status: ',stack['data']['lifecycle-state'])
    print('    Time Created: ', stack['data']['time-created'])
    print('    Stack ID: ',stack['data']['id'])
except:
    print ('Error: ',stdout )
print('\n---------------')
# pprint.pprint(stackPlan)

### Get Stack's State

In [ ]:
file = "\"-\""
stackId = 'ocid1.ormstack.oc1.iad.aaaaaaaavh6aptcq2mwi2x4jh2m3ypohdeyyrjy7td2oqujc2fc73ahk4v6q'
command = 'oci resource-manager stack get-stack-tf-state --stack-id  ' \
          + stackId + ' --file ' + file + config
print('Command: \n', command)
stdout, stderr = runCommand(command)
print(stderr)
try:
    stackStates = json.loads(stdout)
except:
    print ('Error: ',stdout )
for stackState in stackStates['resources'] :  #['resources']['instances']['attributes']:
    for stackStateInstance in stackState['instances']:
#         pprint.pprint (stackState['instances'])
        print(stackStateInstance['attributes']['display_name'])
        print( '   Status: '  ,stackStateInstance['attributes']['state'])
        configDict = stackStateInstance['attributes']['config']
#
        if len(configDict.keys()) > 0:
            print('    Config:')
            [print( '        ' + key,':',value) for key, value in configDict.items()]
        try:
            print( '    Image: '  ,stackStateInstance['attributes']['image'])
        except:
            pass
        print( '    ID: '  ,stackStateInstance['attributes']['id'])
        print('--')
print('\n-------------')
#     pprint.pprint (stackState['instances'])
print('DONE')

### Create Stack from file

<div class="alert alert-block alert-info">
<b>Tip:</b> To Check if Stack creation has completed, re-run the Stack List Report.</div>


In [ ]:
displayName = '\"Speedometer (CostApp.zip) Stack from file\"'
stackDescription = '\"Used CLI to create Stack \"'
configSource = '\"../terraform_files/CostApp.zip\"'
command = 'oci resource-manager  stack create --compartment-id  ' + targetComptId \
          + ' --config-source ' + configSource \
          + ' --description ' + stackDescription \
          + ' --display-name ' + displayName \
          + ' --terraform-version  "1.0.x" ' \
          +  config
print('Command: \n', command)
stdout, stderr = runCommand(command)
print(stderr)
try:
    stackOut = json.loads(stdout)
except:
    print ('Error: ',stdout )

print('Done')

### Create Stack from Private Template

In [ ]:
variables =  '"{\\"Env\\": \\"Blue\\", \\"Log_group_id\\": \\"cecece\\"}"'
displayName = '\"Speedometer Stack 12/23 4pm \"'
sourceWorkingDirectory = '\"terraform_templates/speedometer/environments/Pre-prod_206255/\"'
stackDescription = '\"This Stack included .DS_Store files\"'
templateId = 'ocid1.ormtemplate.oc1.iad.amaaaaaafmyzdhaacgbvkgntyncqz2533escj7ongeomalp6jucsho3jaiia'
command = 'oci resource-manager stack create-from-template ' \
        + ' --compartment-id ' + targetComptId \
        + ' --description ' + stackDescription \
        + ' --display-name ' + displayName \
        + ' --template-id ' + templateId  \
        + ' --terraform-version  "1.0.x" ' \
        + config
#         + ' --variables ' + variables  \
#         + ' --working-directory ' + sourceWorkingDirectory \

print('Command: \n', command)
stdout, stderr = runCommand(command)
print(stderr)
try:
    stackOut = json.loads(stdout)
except:
    print ('Error: ',stdout )

print('Done')

### Create Stack from Github

In [ ]:

sourceRepositoryUrl  =  '\"https://github.com/tr/truccr_oci-iac.git\"'
sourceBranchName = '\"terra2\"'
sourceWorkingDirectory = '\"terraform_templates/speedometer/environments/Pre-prod_206255/\"'
displayName = '\"Speedometer Stack 12/23 \"'
stackDescription = '\"This Stack included .DS_Store files\"'
configSourceProvider = 'ocid1.ormconfigsourceprovider.oc1.iad.aaaaaaaaqvci2yp3mgmkdggokooxgwwxhevnflp4r2yaie5hmmm7kqhibpoq'
command = 'oci resource-manager  stack create-from-git-provider --compartment-id  ' + targetComptId \
          + ' --config-source-configuration-source-provider-id ' + configSourceProvider  \
          + ' --config-source-repository-url ' + sourceRepositoryUrl \
          + ' --config-source-branch-name ' + sourceBranchName \
          + ' --description ' + stackDescription \
          + ' --display-name ' + displayName \
          + ' --config-source-working-directory ' + sourceWorkingDirectory \
          + ' --terraform-version  "1.0.x" ' \
          +  config
 
print('Command: \n', command)
stdout, stderr = runCommand(command)
print(stderr)
try:
    stackOut = json.loads(stdout)
except:
    print ('Error: ',stdout )

print('Done')

### Purge Stack

## Jobs

### Stack Jobs

In [ ]:
stackId = 'ocid1.ormstack.oc1.iad.aaaaaaaaj5shdkrnand75vot25t7fgno67ini2e7jlnt45pkv4yv7swaw5eq'
command = 'oci resource-manager job list  --stack-id ' + stackId  + config
print('Command: \n', command)
stdout, stderr = runCommand(command)
print(stderr)
stackJobs = ''
try:
    stackJobs = json.loads(stdout)
except:
    print( 'Json convert error' ,stdout)

for job in stackJobs['data']:
    print(job['display-name'])
    print('    Operation: ', job['operation'])
    print('    Status: ' , job['lifecycle-state'])
    print('    ID: ', job['id'])
    print('    Time Created: ', job['time-created'])
    print('    Time Finished: ', job['time-finished'])
    print('--')
print('------')
pprint.pprint(stackJobs['data'])
print('Done')

### Stack Plan

In [ ]:
stackId = 'ocid1.ormstack.oc1.iad.aaaaaaaa6hfu2o6gvmvpigaqj5k7crnhnpifm3cqqpcnhq6nly53yzcdssgq'
displayName = '\"TEST  ** Plan  changes with variables \"'
variables =  '"{\\"env\\": \\"Blue\\", \\"log_group_id\\": \\"cecece\\"}"'
command = ' oci resource-manager job create-plan-job --stack-id  ' + stackId \
        + ' --display-name ' + displayName  \
        + ' --variables ' + variables \
        +  config
print('Command: \n', command)
stdout, stderr = runCommand(command)
print(stderr)
print('Creation of Plan Job Completed!!')
try:
    stackPlan = json.loads(stdout)
    print(stackPlan['data']['display-name'])
    print('    Git Branch: ',stackPlan['data']['config-source']['branch-name'])
    print('    Git Directory: ',stackPlan['data']['working-directory'])
    print('    Terraform Operation: ',stackPlan['data']['operation'])
    print('    Status: ',stackPlan['data']['lifecycle-state'])
    print('    Time Created: ', stackPlan['data']['time-created'])
    print('    Job ID: ',stackPlan['data']['id'])
    print('    Stack ID: ',stackPlan['data']['stack-id'])
except:
    print ('Error: ',stdout )
print('\n---------------')
# pprint.pprint(stackPlan)
print('Done')

### Stack Apply

In [ ]:
displayName = '\"Test for Jupyter Scripts \"'
execPlanStrategy = 'AUTO_APPROVED'
# stackId = 'ocid1.ormstack.oc1.iad.aaaaaaaa3bnekl6352t6cfwnzkener6zddvdbsqis5fupd6ucm4aghxnh2uq'
command = ' oci resource-manager job create-apply-job --stack-id  ' + stackId \
          + ' --display-name ' + displayName  \
          + ' --execution-plan-strategy ' + execPlanStrategy  \
          +  config
print('Command: \n', command)
stdout, stderr = runCommand(command)
print(stderr)
try:
    stackPlan = json.loads(stdout)
    print(stackPlan['data']['display-name'])
    print('    Git Branch: ',stackPlan['data']['config-source']['branch-name'])
    print('    Git Directory: ',stackPlan['data']['working-directory'])
    print('    Terraform Operation: ',stackPlan['data']['operation'])
    print('    Status: ',stackPlan['data']['lifecycle-state'])
except:
    print ('Error: ',stdout )
print('Done')



### Stack Destroy

<div class="alert alert-block alert-danger">
<b>Danger:</b> Running Stack Destroy will permanently remove OCI components.  Use caution when using.
</div>

In [ ]:
execPlanStrategy = 'AUTO_APPROVED'
stackId = 'ocid1.ormstack.oc1.iad.aaaaaaaaj5shdkrnand75vot25t7fgno67ini2e7jlnt45pkv4yv7swaw5eq'
command = ' oci resource-manager job create-destroy-job --stack-id  ' \
           + stackId  +  ' --execution-plan-strategy '  + execPlanStrategy  +     config
print('Do you want to destroy?(yes/no): ', stackId)
a = input()
if a == 'yes':
    print('yessire')
    print('Command: \n', command)
    stdout, stderr = runCommand(command)
    print(stderr)
    print(stdout)
print('Destroy Stack Cancelled !')

### Purge Jobs

**TBD**

## Templates

### PrivateTemplate List

In [ ]:
sortBy = ' --sort-by TIMECREATED --sort-order DESC  '
command = 'oci resource-manager template list --template-category-id 3 -c ' \
          + targetComptId + sortBy + config
print('Command: \n', command)
stdout, stderr = runCommand(command)
print(stderr)
templates = json.loads(stdout)

print('There were ' + str(len(templates['data']['items']) )+ '  Templates Found\n -------')
for template in templates['data']['items']:
    print(template['display-name'])
    print('    Description: ', template['description'] )
    print('    ID: ', template['id']  + '\n ---')
# pprint.pprint(templates['data'])

### Get Template

In [ ]:
templateId = \
'ocid1.ormtemplate.oc1.iad.amaaaaaafmyzdhaacgbvkgntyncqz2533escj7ongeomalp6jucsho3jaiia'
command = 'oci resource-manager template get --template-id  ' + templateId  + config
print('Command: \n', command)
stdout, stderr = runCommand(command)
print(stderr)
template = json.loads(stdout)

pprint.pprint(template['data'])

### Get Template Config File

In [ ]:

file = "\"templatex.zip\""
templateId = 'ocid1.ormtemplate.oc1.iad.amaaaaaafmyzdhaahpmtuar3i47j24kqhbcice7u3rcjk3pn32jug7xah3aq'
command = ' oci resource-manager template get-template-tf-config --template-id '+ templateId  + ' --file ' + file + config
print('Command: \n', command)
stdout,stderr = runCommand(command)
print(stderr)
try:
    template = json.loads(stdout)
except:
    print ('Error: ',stdout )

print('Done')

### Create Private Template

In [ ]:
# config1 = from_file(file_location="")
import base64
with open( r"CostApp.zip",'rb' ) as file:
    zipContents = file.read()
encodedZip = base64.b64encode(zipContents).decode('ascii')

def get_signer(securityProfile):
    securityConfig = oci.config.from_file(profile_name=securityProfile)
    token_file = securityConfig['security_token_file']
    token = None
    with open(token_file, 'r') as f:
         token = f.read()    
    private_key = oci.signer.load_private_key_from_file(securityConfig['key_file'])
    return oci.auth.signers.SecurityTokenSigner(token, private_key) 

tenancyProfile = 'tr-ash'
tenancyConfig = oci.config.from_file(profile_name=tenancyProfile)

zipfileupload = oci.resource_manager.models.CreateTemplateZipUploadConfigSourceDetails()
zipfileupload.config_source_type = 'ZIP_UPLOAD'
# zipfileupload.working_directory = 'MyTerraformScripts'
zipfileupload.zip_file_base64_encoded = encodedZip

resourcemanager = oci.resource_manager.models.CreateTemplateDetails()
resourcemanager.compartment_id = tenancyConfig['target_compartment']
resourcemanager.display_name = "Sample Template2"
resourcemanager.description = "Sample Temaplate2"
resourcemanager.template_config_source = zipfileupload

# def getCommandOutput(pCommand):
# Stack = oci.resource_manager.ResourceManagerClient(config1)
Template = oci.resource_manager.ResourceManagerClient({'region': tenancyConfig['region']},
                                          signer=get_signer(tenancyConfig['auth_profile'] ))

Template.create_template(create_template_details=resourcemanager)


### Update Private Template

In [ ]:
import base64
with open( r"CostApp.zip",'rb' ) as file:
    zipContents = file.read()
encodedZip = base64.b64encode(zipContents).decode('ascii')
zipfileupload = oci.resource_manager.models.UpdateTemplateZipUploadConfigSourceDetails()
zipfileupload.config_source_type = 'ZIP_UPLOAD'
# zipfileupload.working_directory = 'MyTerraformScripts'
zipfileupload.zip_file_base64_encoded = encodedZip
resourcemanager = oci.resource_manager.models.UpdateTemplateDetails()
resourcemanager.compartment_id = tenancyConfig['target_compartment']
resourcemanager.display_name = "Sample Template2"
resourcemanager.description = "Sample Temaplate2x"
resourcemanager.
resourcemanager.template_config_source = zipfileupload
Template = oci.resource_manager.ResourceManagerClient({'region': tenancyConfig['region']},
                                          signer=get_signer(tenancyConfig['auth_profile'] ))

testxx = Template.update_template(update_template_details=resourcemanager)
print(testxx.data)

### Create Stack from Template

In [ ]:

stackTemplateDetails=  oci.resource_manager.models.CreateStackTemplateConfigSourceDetails()
stackTemplateDetails.template_id=  \
   'ocid1.ormtemplate.oc1.iad.amaaaaaafmyzdhaacgbvkgntyncqz2533escj7ongeomalp6jucsho3jaiia'
stackTemplateDetails.config_source_type="TEMPLATE_CONFIG_SOURCE"
stackTemplateDetails.working_directory='terraform_templates'
# stackTemplateDetails.config_source_type='UNKOWN'

createStackDetails=oci.resource_manager.models.CreateStackDetails()
createStackDetails.config_source=stackTemplateDetails
createStackDetails.compartment_id = tenancyConfig['target_compartment']
createStackDetails.display_name = "Stack from Sample Template"
createStackDetails.description = "Stack from Sample Temaplate"
createStackDetails.terraform_version =  "1.0.x"
createStackDetails.variables={
            'env': 'xme',
            'log_group_id': 'wXfZ36ni34dituFB6MBo' }

stackTemplate = oci.resource_manager.ResourceManagerClient({'region': tenancyConfig['region']},
                                          signer=get_signer(tenancyConfig['auth_profile'] ))

create_stack_response=stackTemplate.create_stack(create_stack_details=createStackDetails)
# Get the data from response
print(create_stack_response.data)

